In [1]:
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai2.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [2]:
from sklearn.tree import export_graphviz

def draw_tree(t, df, size=10, ratio=0.6, precision=0, **kwargs):
    s=export_graphviz(t, out_file=None, feature_names=df.columns, filled=True, rounded=True,
                      special_characters=True, rotate=False, precision=precision, **kwargs)
    return graphviz.Source(re.sub('Tree {', f'Tree {{ size={size}; ratio={ratio}', s))

In [4]:
from scipy.cluster import hierarchy as hc

def cluster_columns(df, figsize=(10,6), font_size=12):
    corr = np.round(scipy.stats.spearmanr(df).correlation, 4)
    corr_condensed = hc.distance.squareform(1-corr)
    z = hc.linkage(corr_condensed, method='average')
    fig = plt.figure(figsize=figsize)
    hc.dendrogram(z, labels=df.columns, orientation='left', leaf_font_size=font_size)
    plt.show()

In [10]:
# Récupération des données depuis une compétition Kaggle
creds = '{"username":"eckmoule","key":"f190a59522c01d88921517a8b4c37f37"}'
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write(creds)
    cred_path.chmod(0o600)
path = URLs.path('bluebook')
path, cred_path

(Path('/storage/archive/bluebook'), Path('/root/.kaggle/kaggle.json'))

In [12]:
from kaggle import api


api.competition_download_cli('bluebook-for-bulldozers', path=path)
file_extract(path/'bluebook-for-bulldozers.zip')
path.ls(file_type='text')

bluebook-for-bulldozers.zip: Skipping, found more recently modified local copy (use --force to force download)


(#7) [Path('/storage/archive/bluebook/Test.csv'),Path('/storage/archive/bluebook/random_forest_benchmark_test.csv'),Path('/storage/archive/bluebook/ValidSolution.csv'),Path('/storage/archive/bluebook/TrainAndValid.csv'),Path('/storage/archive/bluebook/median_benchmark.csv'),Path('/storage/archive/bluebook/Valid.csv'),Path('/storage/archive/bluebook/Machine_Appendix.csv')]

In [47]:
# Exploration des données 
%time df = pd.read_csv(path/'TrainAndValid.csv', low_memory=False)
df.columns, len(df)

CPU times: user 3.18 s, sys: 380 ms, total: 3.56 s
Wall time: 3.56 s


(Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
        'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
        'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
        'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
        'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
        'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
        'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
        'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
        'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
        'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
        'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
        'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
        'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
       dtype='object'),
 412698)

In [16]:
df["SalePrice"].head()

0    66000.0
1    57000.0
2    10000.0
3    38500.0
4    11000.0
Name: SalePrice, dtype: float64

In [17]:
df["SalePrice"].describe()

count    412698.000000
mean      31215.181414
std       23141.743695
min        4750.000000
25%       14500.000000
50%       24000.000000
75%       40000.000000
max      142000.000000
Name: SalePrice, dtype: float64

In [18]:
df["ProductSize"].head()

0       NaN
1    Medium
2       NaN
3     Small
4       NaN
Name: ProductSize, dtype: object

In [19]:
df["ProductSize"].unique()

array([nan, 'Medium', 'Small', 'Large / Medium', 'Mini', 'Large',
       'Compact'], dtype=object)

In [20]:
df["ProductSize"].memory_usage()/1024

3224.328125

In [48]:
# On transforme la chaine de caractère répétée par un nombre (id categorie)
df["ProductSize"] = df["ProductSize"].astype('category')
df["ProductSize"].memory_usage()/1024

403.353515625

In [23]:
df["ProductSize"].cat.categories

Index(['Compact', 'Large', 'Large / Medium', 'Medium', 'Mini', 'Small'], dtype='object')

In [49]:
# Gestion des dates. On utilise une méthode de fastAI qui va créer des colonnes indicatives sur la date (fin de mois,
# années, jour de la semaine, ...) qui seront plus pertinantes que la date brutes pour notre modèle
df = add_datepart(df, 'saledate')
df.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',


In [50]:
# On va chercher a trouver le log du prix et non le prix car la compétition Kaggle va calculer l'erreur sur le ratio
# le log du prix trouvé / prévu. Cela afin d'améliorer la métrique (1000 d'erreur sur 100.000 ou sur 5.000 n'est pas la 
# même erreur). 

dep_var = 'SalePrice'
df[dep_var] = np.log(df[dep_var])
df[dep_var].describe()

count    412698.000000
mean         10.105977
std           0.695005
min           8.465900
25%           9.581904
50%          10.085809
75%          10.596635
max          11.863582
Name: SalePrice, dtype: float64

In [51]:
df_test = pd.read_csv(path/'Test.csv', low_memory=False)
df_test = add_datepart(df_test, 'saledate')

In [52]:
# On sépare les données d'entrainement et de validation.
# On va prendre les données les plus récentes du pour la validation. En effet l'objectif du modèle est de déterminer les 
# prix futurs à partir des prix passés. 

cond = (df.saleYear<2011) | (df.saleMonth<10) 
train_idx = np.where( cond)[0]
valid_idx = np.where(~cond)[0]

splits = (list(train_idx), list(valid_idx))

len(train_idx), len(valid_idx)

(404710, 7988)

In [53]:
# FastAI propose une fonction pour séparer les colonnes continues des colonnes de type catégorie

cont,cat = cont_cat_split(df, 1, dep_var=dep_var) # dep_var = SalePrice on exclue cette colonne résultat
cont[:5],cat[:5]

(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID'],
 ['UsageBand',
  'fiModelDesc',
  'fiBaseModel',
  'fiSecondaryDesc',
  'fiModelSeries'])

In [54]:
# Les tabular procs permettent d'appliquer une modification sur l'ensemble des colonnes. 
# Ici categorify va créer des catégories (ID de catégorgies) pour toutes les colonnes de ce type et FillMissing va remplacer
# les données manquantes par la moyenne puis rajouter une colonne indiquant que la données était manquante.
procs = [ Categorify, FillMissing]
to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)

In [55]:
len(to.train), len(to.valid)

(404710, 7988)

In [56]:
# On peut sauvegarder la préparation des données 
(path/'to.pkl').save(to)

In [57]:
%time to = (path/'to.pkl').load()

CPU times: user 30.7 ms, sys: 14.7 ms, total: 45.4 ms
Wall time: 48.3 ms


In [58]:
len(to.train), len(to.valid)

(404710, 7988)